# VacationPy

## Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,12.08,89,10,2.67,AU,1686093479
1,1,waitangi,-43.9535,-176.5597,11.57,91,100,0.89,NZ,1686093479
2,2,cabedelo,-6.9811,-34.8339,25.37,82,20,4.43,BR,1686093282
3,3,pizarro,4.9533,-77.3660,26.72,83,100,2.43,CO,1686093282
4,4,grytviken,-54.2811,-36.5092,2.50,96,90,2.04,GS,1686093447


## Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [18]:
# Create the scatter plot using hvplot.pandas
plot = city_data_df.hvplot.points(x='Lng', y='Lat', c='Humidity', size='Humidity', cmap='Blues', alpha=0.8)

# Set plot properties
plot = plot.opts(width=600, height=400, title='City Map: Humidity', colorbar=True, xlabel='Longitude', ylabel='Latitude')

# Display the plot
plot

:Points   [Lng,Lat]   (Humidity)

## Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities based on criteria
filtered_df = city_data_df[(city_data_df['Max Temp'] > 25) & (city_data_df['Humidity'] < 50) & (city_data_df['Cloudiness'] == 0)]

# Drop rows with null values
filtered_df = filtered_df.dropna()

filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,fort bragg,35.1390,-79.0060,29.34,42,0,4.63,US,1686093480
41,41,taoudenni,22.6783,-3.9836,34.27,15,0,3.40,ML,1686093287
49,49,oak ridge,36.0104,-84.2696,29.62,45,0,1.54,US,1686093288
85,85,summerland,49.5998,-119.6691,28.94,18,0,6.17,CA,1686093293
117,117,daulatpur,26.5015,67.9706,33.92,40,0,2.47,PK,1686093298
126,126,baker city,44.7749,-117.8344,28.84,35,0,3.60,US,1686093299
164,164,merritt,50.1123,-120.7942,27.96,13,0,1.79,CA,1686093502
189,189,anar darah,32.7587,61.6540,26.54,14,0,3.84,AF,1686093308
371,371,turpan,42.9333,89.1667,30.96,13,0,0.84,CN,1686093529
405,405,brookings,44.3114,-96.7984,31.77,46,0,5.14,US,1686093341


In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column named "Hotel Name"
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,89,
1,waitangi,NZ,-43.9535,-176.5597,91,
2,cabedelo,BR,-6.9811,-34.8339,82,
3,pizarro,CO,4.9533,-77.3660,83,
4,grytviken,GS,-54.2811,-36.5092,96,


## For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000  # radius in meters
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    data = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = data["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
waitangi - nearest hotel: Hotel Chathams
cabedelo - nearest hotel: No hotel found
pizarro - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
cromer - nearest hotel: Hotel de Paris
howard springs - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
utrik - nearest hotel: No hotel found
kresttsy - nearest hotel: No hotel found
saudarkrokur - nearest hotel: Mikligarður
port-aux-francais - nearest hotel: Keravel
puerto natales - nearest hotel: Dorotea Patagonia Hostel
sirte - nearest hotel: فندق المدينة-سرت
fort bragg - nearest hotel: Airborne Inn Lodging
anadyr - nearest hotel: Гостевой дом
guerrero negro - nearest hotel: Plaza sal paraiso
thompson - nearest hotel: Thompson Inn
albany - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
hithadhoo - nearest hotel: Pebbles Inn
jamestown - nearest hotel: DoubleTree Jamestown
la sarre - nearest hotel: No hotel found
bredasdo

anar darah - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
selfoss - nearest hotel: Miðgarður
tolanaro - nearest hotel: Hôtel Mahavokey
qui nhon - nearest hotel: No hotel found
qarauyl - nearest hotel: No hotel found
sitka - nearest hotel: Aspen Suites Hotel
dali - nearest hotel: 风花雪月大酒店
tempio pausania - nearest hotel: Park Hotel
chibougamau - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: Casa de Olas
kill devil hills - nearest hotel: Mariner Days Inn & Suites
faya-largeau - nearest hotel: No hotel found
ola - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
vadso - nearest hotel: Scandic Vadsø
elliot - nearest hotel: No hotel found
christchurch - nearest hotel: Ibis Hotel Christchurch
marsa alam - nearest hotel: Blue House Hotel
yambio - nearest hotel: Kabash Inn
wani - nearest hotel: Hotel lords and bar
kavaratti - nearest hotel: No hotel found
tukrah - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel

budva - nearest hotel: Dimic Ellite
sardinata - nearest hotel: El Prado
santanyi - nearest hotel: Santanyí
aripuana - nearest hotel: No hotel found
fortaleza - nearest hotel: Hotel Catedral
nieuw nickerie - nearest hotel: Hotel Pak Hap
hau-ri - nearest hotel: No hotel found
plymstock - nearest hotel: Hotel Mount Batten
mizdah - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
santa ana - nearest hotel: Holiday Inn Express
berezovo - nearest hotel: Град Березов
alofi - nearest hotel: Matavai Resort
lepsy - nearest hotel: Orley
minna - nearest hotel: Haske Main Hotel
marathon - nearest hotel: Rib Mountain Inn
holualoa - nearest hotel: Kona Hotel
kingscliff - nearest hotel: Kingscliff Beach Hotel
ivdel' - nearest hotel: гостиница Ивдель
keelung - nearest hotel: 華國商務飯店
agde - nearest hotel: Hôtel Le Languedoc
brookings - nearest hotel: Quality Inn
turkmenbasy - nearest hotel: Türkmenbaşy Oteli
pingdingshan - nearest hotel: No hotel found
thunder bay - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,89,Villa Howden
1,waitangi,NZ,-43.9535,-176.5597,91,Hotel Chathams
2,cabedelo,BR,-6.9811,-34.8339,82,No hotel found
3,pizarro,CO,4.9533,-77.3660,83,No hotel found
4,grytviken,GS,-54.2811,-36.5092,96,No hotel found
...,...,...,...,...,...,...
567,vuktyl,RU,63.8567,57.3094,81,Вуктыл
568,ciudad melchor muzquiz,MX,27.8833,-101.5167,38,No hotel found
569,san andres,CO,12.5847,-81.7006,73,Hotel Decameron Isleño
570,mahina,PF,-17.5065,-149.4890,54,Motu Martin


## Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
# Create the scatter plot using hvplot.pandas
plot = hotel_df.hvplot.points(x='Lng', y='Lat', c='Humidity', size='Humidity', cmap='Blues', alpha=0.8,
                                  hover_cols=['City', 'Country', 'Hotel Name'])

# Set plot properties
plot = plot.opts(width=600, height=400, title='City Map: Humidity', colorbar=True, xlabel='Longitude', ylabel='Latitude')

# Display the plot
plot

:Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)